# Trusting your data using the Agents SDK and the dbt MCP server

Many AI workflows require the ability to access and query structured data in a database. 

[dbt](https://www.getdbt.com/product/what-is-dbt) is the modern standard for data transformation. The dbt framework and platform are used by thousands of teams wanting to scale their data lifecycle leveraging the [Analytics Developer Lifecycle (ADLC)](https://www.getdbt.com/resources/the-analytics-development-lifecycle) . With the ADLC, developers can plan, develop, test, document, and deploy trusted data assets including metrics.

You can use OpenAI APIs on top of [the dbt MCP server](https://docs.getdbt.com/docs/dbt-ai/about-mcp) to:

- Discover which assets exist in your dbt project and metadata about them (descriptions, columns etc)
- Accurately and consistently query your most important metrics using the dbt Semantic Layer

The dbt MCP server is available [both locally hosted and remotely hosted via dbt Platform](https://docs.getdbt.com/docs/dbt-ai/about-mcp). The following demonstration uses the remote MCP server. To follow along with the examples, you can use your existing dbt account if you have one, or if not, you can set one up with a sample jaffle shop dataset following instructions in [this repo](https://github.com/dbt-labs/jaffle-shop).

## 1. Data Discovery with Agents SDK + dbt MCP server

Before we ask an LLM to write SQL, it needs a reliable map of the warehouse. In this section, you'll connect OpenAI's Agents SDK to the **dbt MCP server** so the agent can *browse your trusted dbt assets*—not raw schemas or ad-hoc tables. Using dbt Cloud's metadata (models, descriptions, columns, and lineage), the agent can list marts, inspect model details, and trace dependencies to answer questions like "what should I query to learn about my customers?"

![Data Discovery Process Flow](../images/trusted_data_process_flow_openai_dbt_mcp.png)

*The diagram above shows how the OpenAI Agents SDK connects to the dbt MCP server for data discovery.*

To call the remote MCP server from the OpenAI Agents SDK, we need to install the SDK via pip in a Python virtual environment by running the following:

In [ ]:
!pip install openai-agents

We also need to set the following environment variables

| **Variable** | **Description** |
| --- | --- |
| `OPENAI_API_KEY` | Your OpenAI API key |
| `DBT_PROD_ENV_ID` | The environment ID of your Production environment. When connected to the dbt platform, you can see it in the URL |
| `DBT_TOKEN` | A service token or personal access token with access to the dbt environment |
| `DBT_HOST` | The host of your dbt account if different from `cloud.getdbt.com` |

The following code allows the OpenAI agents SDK to connect to the remote dbt MCP server and to leverage the following data discovery tools

- `get_mart_models` - Gets all mart models
- `get_all_models` - Gets all models
- `get_model_details` - Gets details for a specific model
- `get_model_parents` - Gets parent nodes of a specific model
- `get_model_children` - Gets children models of a specific model

In [ ]:
import asyncio
import os

from agents import Agent, Runner, trace
from agents.mcp import create_static_tool_filter
from agents.mcp.server import MCPServerStreamableHttp

async def main():
    prod_environment_id = os.environ.get("DBT_PROD_ENV_ID", os.getenv("DBT_ENV_ID"))
    token = os.environ.get("DBT_TOKEN")
    host = os.environ.get("DBT_HOST", "cloud.getdbt.com")

    async with MCPServerStreamableHttp(
        name="dbt",
        params={
            "url": f"https://{host}/api/ai/v1/mcp/",
            "headers": {
                "Authorization": f"token {token}",
                "x-dbt-prod-environment-id": prod_environment_id,
            },
        },
        client_session_timeout_seconds=20,
        cache_tools_list=True,
        tool_filter=create_static_tool_filter(
            allowed_tool_names=[
                "get_mart_models",
                "get_all_models",
                "get_model_details",
                "get_model_parents",
                "get_model_children",
            ],
        ),
    ) as server:
        agent = Agent(
            name="Assistant",
            instructions="Use the tools to answer the user's questions",
            mcp_servers=[server],
            model="gpt-5", 
        )
        with trace(workflow_name="Conversation"):
            conversation = []
            result = None
            while True:
                if result:
                    conversation = result.to_input_list()
                conversation.append({"role": "user", "content": input("User > ")})
                result = await Runner.run(agent, conversation)
                print(result.final_output)

if __name__ == "__main__":
    try:
        asyncio.run(main())
    except KeyboardInterrupt:
        print("\nExiting.")

If we run `python openai_agents_dbt.py` , we can then start a conversation with the LLM and get information about our data.

> User > What are my marts models in my warehouse?
> 

> Answer:
> 

```bash
1. **Customers**
   - **Unique ID:** model.jaffle_semantic_layer_testing.customers
   - **Description:** Customer overview data mart, offering key details for each unique customer. One row per customer.

2. **Order Items**
   - **Unique ID:** model.jaffle_semantic_layer_testing.order_items
   - **Description:** (No description provided)

3. **Orders**
   - **Unique ID:** model.jaffle_semantic_layer_testing.orders
   - **Description:** Order overview data mart, offering key details for each order including if it's a customer's first order and a food vs. drink item breakdown. One row per order.
```

This response returns detailed information about the models in our project which are best suited for user consumption (marts).

We can continue the conversation to get more details about specific data assets.

> User > Give me more information about the Customers table
> 

> Answer:
> 

```
Here's more information about the **Customers** table:

### Description
Customer overview data mart, offering key details for each unique customer. One row per customer.

### Database Details
- **Database:** analytics
- **Schema:** analytics

### Compiled SQL
The `Customers` model is built with several CTEs that pull from staging tables and aggregate data:

```sql
with
customers as (
    select * from analytics.analytics.stg_customers
),
orders_mart as (
    select * from analytics.analytics.orders
),
order_items_mart as (
    select * from analytics.analytics.order_items
),
order_summary as (
    select
        customer_id,
        count(distinct om.order_id) as count_lifetime_orders,
        count(distinct om.order_id) > 1 as is_repeat_buyer,
        min(om.ordered_at) as first_ordered_at,
        max(om.ordered_at) as last_ordered_at,
        sum(oi.subtotal) as lifetime_spend_pretax,
        sum(om.order_total) as lifetime_spend
    from orders_mart om
    left join order_items_mart oi on om.order_id = oi.order_id
    group by 1
),
joined as (
    select
        customers.*,
        order_summary.count_lifetime_orders,
        order_summary.first_ordered_at,
        order_summary.last_ordered_at,
        order_summary.lifetime_spend_pretax,
        order_summary.lifetime_spend,
        case
            when order_summary.is_repeat_buyer then 'returning'
            else 'new'
        end as customer_type
    from customers
    left join order_summary
        on customers.customer_id = order_summary.customer_id
)
select * from joined
```

### Columns
- **CUSTOMER_ID** (TEXT): The unique key of the orders mart.
- **CUSTOMER_NAME** (TEXT): Customers' full name.
- **COUNT_LIFETIME_ORDERS** (NUMBER): Total number of orders a customer has ever placed.
- **FIRST_ORDERED_AT** (TIMESTAMP_NTZ): The timestamp when a customer placed their first order.
- **LAST_ORDERED_AT** (TIMESTAMP_NTZ): The timestamp of a customer's most recent order.
- **LIFETIME_SPEND_PRETAX** (NUMBER): The sum of all the pre-tax subtotals of every order a customer has placed.
- **LIFETIME_SPEND** (NUMBER): The sum of all the order totals (including tax) that a customer has ever placed.
- **CUSTOMER_TYPE** (TEXT): Options are 'new' or 'returning', indicating if a customer has ordered more than once or has only placed their first order to date.
```

This answer lists all the columns contained in our table as well as their descriptions. This information can be useful to understand the actual definition of what information is stored in which column.

Finally, we can ask another question to the LLM to understand the lineage of the `orders` table.

> User > What tables do the `orders` table get information from?
> 

> Answer:
> 

```bash
The **Orders** table gets its information from the following tables:

1. **stg_order_items**
   - **Description:** Individual food and drink items that make up our orders, one row per item.

2. **stg_orders**
   - **Description:** Order data with basic cleaning and transformation applied, one row per order.

3. **stg_products**
   - **Description:** Product (food and drink items that can be ordered) data with basic cleaning and transformation applied, one row per product.

4. **stg_supplies**
   - **Description:** List of our supply expenses data with basic cleaning and transformation applied. One row per supply cost, not per supply. As supply costs fluctuate, they receive a new row with a new UUID, resulting in multiple rows per supply_id.
```

This answer shows the `orders` table actually merges data related to orders, order items, products, and supplies.

## 2. Querying governed metrics with the dbt Semantic Layer

Now that the agent can *find* trusted data, let's help it *answer questions* with consistency. In this section, we connect OpenAI's Agents SDK to the **dbt Semantic Layer via the MCP server**, so natural-language questions resolve to governed metrics—not ad-hoc SQL. The agent discovers available metrics and dimensions, generates a **Semantic Layer query**, and SL deterministically compiles it to valid warehouse SQL. You get the same answer every time, aligned to your definitions, with the option to inspect the compiled SQL for transparency.

A common approach to query data warehouses from interacting with AI is to have the LLM generate SQL based on user inputs in natural language. This is usually described as text-to-sql.

In some cases, the LLM will generate the exact query that is required to answer the user question, but there are occasions where either the SQL query generated might not be valid or that it will return incorrect data that doesn't conform with the user expectation and definition of some metrics.

What could be happening to cause this:

- the resulting query might be joining tables that aren't supposed to be joined together
- the SQL query generated does not comply with the data warehouse syntax
- the LLM generates different queries for the same question
- it is difficult for users to query specific metrics in natural language with their business specific logic. Example: The team thinks of new customers with the exclusion of trial customers

![Traditional Text-to-SQL Process](../images/trusted_data_process_flow_openai_data_warehouse.png)

*This diagram illustrates the traditional text-to-SQL approach where the LLM directly generates SQL queries for the data warehouse.*

While you can  improve the Text to SQL flow by providing additional context to the LLM there are some circumstances where pairing a deterministic system (a Semantic Layer) is preferable.

### An introduction to the dbt Semantic Layer

The dbt Semantic Layer lets you define and query metrics and measures on transformed warehouse data.

To define metrics, users add specific YAML configurations to their dbt project. The metrics will include information about joining data across tables and are version controlled in git, like any dbt component. The specification is flexible enough to define exactly how a metric should be calculated. For examples, you could define a "number of paid customers" metric as the count of rows in the table `dim_customers` where the customer didn't cancel during the trial period.

Once metrics are defined and agreed upon, the dbt Semantic Layer can be queried directly, without the need to specify what tables to get data from, or what SQL to write. The SQL is dynamically generated by the Semantic Layer according to the configuration. The metrics are accessible [calling  APIs](https://docs.getdbt.com/docs/use-dbt-semantic-layer/consume-metrics#query-with-apis) or by using [BI tools with built-in integrations with the dbt Semantic Layer](https://docs.getdbt.com/docs/cloud-integrations/avail-sl-integrations). 

In the past, the integration with BI tools was the primary use case for the Semantic layer, but increasingly the Semantic Layer is used in AI workflows.

Below is a short example showing how to define measures in the Semantic Layer.

```yaml
semantic_models:
  - name: orders
    defaults:
      agg_time_dimension: order_date
    description: Order fact table. This table's grain is one row per order.
    model: ref('fct_orders')
    entities:
      - name: order_id
        type: primary
      - name: customer
        expr: customer_id
        type: foreign
    dimensions:
      - name: order_date
        type: time
        type_params:
          time_granularity: day
    measures:   
      - name: order_total
        description: The total amount for each order including taxes.
        agg: sum
        expr: amount
      - name: order_count
        expr: 1
        agg: sum
      - name: customers_with_orders
        description: Distinct count of customers placing orders
        agg: count_distinct
        expr: customer_id
```

### Integrating the Agents SDK and the dbt Semantic Layer via the dbt MCP server

The Semantic Layer can serve as an intermediary, deterministic step between the agent and the database.

![Semantic Layer Integration Process](../images/trusted_data_process_flow_openai_dbt_semantic_layer.png)

*This diagram shows the improved approach using the dbt Semantic Layer as an intermediary between OpenAI and the data warehouse, ensuring consistent and governed metrics.*

To create this flow, leveraging the OpenAI Agents SDK and GPT 5, we need to install the `openai-agents` package

In [ ]:
!pip install openai-agents

and set the following environment variables

| **Variable** | **Description** |
| --- | --- |
| `OPENAI_API_KEY` | Your OpenAI API key |
| `DBT_PROD_ENV_ID` | The environment ID of your Production environment. When connected to the dbt platform, you can see it in the URL |
| `DBT_TOKEN` | A service token or personal access token with access to the dbt environment |
| `DBT_HOST` | The host of your dbt account if different from `cloud.getdbt.com` |

Copy/paste this code locally in `openai_dbt_semantic_layer.py` and run `python openai_dbt_semantic_layer.py` to interact with OpenAI and get results from the dbt Semantic Layer.

In this example, we connect again to the dbt MCP server but we activate a different set of tools:

- `list_metrics` - Retrieves all defined metrics
- `get_dimensions` - Gets dimensions associated with specified metrics
- `get_entities` - Gets entities associated with specified metrics
- `query_metrics` - Queries metrics with optional grouping, ordering, filtering, and limiting
- `get_metrics_compiled_sql` - Gets and returns the compiled SQL that would be generated for specified metrics and groupings without executing the query

By inspecting `stream_events()` we capture the tools being called, including which  parameters are sent, and print them in the terminal.

In [ ]:
import asyncio
import os

from agents import Agent, Runner, trace
from agents.mcp import create_static_tool_filter
from agents.mcp.server import MCPServerStreamableHttp
from agents.stream_events import RawResponsesStreamEvent, RunItemStreamEvent
from openai.types.responses import ResponseCompletedEvent, ResponseOutputMessage

def print_tool_call(tool_name, params, color="yellow", show_params=True):
    # Define color codes for different colors
    # we could use a library like colorama but this avoids adding a dependency
    color_codes = {
        "grey": "\033[37m",
        "yellow": "\033[93m",
    }
    color_code_reset = "\033[0m"

    color_code = color_codes.get(color, color_codes["yellow"])
    msg = f"Calling the tool {tool_name}"
    if show_params:
        msg += f" with params {params}"
    print(f"{color_code}# {msg}{color_code_reset}")

def handle_event_printing(event, show_tool_calls=True):
    if type(event) == RunItemStreamEvent and show_tool_calls:
        if event.name == "tool_called":
            print_tool_call(
                event.item.raw_item.name,
                event.item.raw_item.arguments,
                color="grey",
                show_params=True,
            )

    if type(event) == RawResponsesStreamEvent:
        if type(event.data) == ResponseCompletedEvent:
            for output in event.data.response.output:
                if type(output) == ResponseOutputMessage:
                    print(output.content[0].text)

async def main():
    prod_environment_id = os.environ.get("DBT_PROD_ENV_ID", os.getenv("DBT_ENV_ID"))
    token = os.environ.get("DBT_TOKEN")
    host = os.environ.get("DBT_HOST", "cloud.getdbt.com")

    async with MCPServerStreamableHttp(
        name="dbt",
        params={
            "url": f"https://{host}/api/ai/v1/mcp/",
            "headers": {
                "Authorization": f"token {token}",
                "x-dbt-prod-environment-id": prod_environment_id,
            },
        },
        client_session_timeout_seconds=20,
        cache_tools_list=True,
        tool_filter=create_static_tool_filter(
            allowed_tool_names=[
                "list_metrics",
                "get_dimensions",
                "get_entities",
                "query_metrics",
                "get_metrics_compiled_sql",
            ],
        ),
    ) as server:
        agent = Agent(
            name="Assistant",
            instructions="Use the tools to answer the user's questions. Do not invent data or sample data.",
            mcp_servers=[server],
            model="gpt-5",
        )
        with trace(workflow_name="Conversation"):
            conversation = []
            result = None
            while True:
                if result:
                    conversation = result.to_input_list()
                conversation.append({"role": "user", "content": input("User > ")})

                async for event in Runner.run_streamed(
                    agent, conversation
                ).stream_events():
                    handle_event_printing(event, show_tool_calls=True)

if __name__ == "__main__":
    try:
        asyncio.run(main())
    except KeyboardInterrupt:
        print("\nExiting.")

Example user interaction: 

> User > What is the revenue per month?
> 

> Answer:
> 

```
# Calling the tool list_metrics with params {}
# Calling the tool get_dimensions with params {"metrics":["revenue"]}
# Calling the tool query_metrics with params {"metrics":["revenue"],"group_by":[{"name":"metric_time","type":"time_dimension","grain":"MONTH"}],"order_by":[{"name":"metric_time","descending":true}],"limit":12}

Here are the monthly revenues for the last 12 months (latest first):

- 2025-08: 102,379
- 2025-07: 90,396
- 2025-06: 93,683
- 2025-05: 91,388
- 2025-04: 79,246
- 2025-03: 70,218
- 2025-02: 40,906
- 2025-01: 42,472
- 2024-12: 33,660
- 2024-11: 26,338
- 2024-10: 20,684
- 2024-09: 17,032
```

> User > What dimensions can we slice the revenue by?
> 

> Answer:
> 

```
# Calling the tool list_metrics with params {}
# Calling the tool get_dimensions with params {"metrics":["revenue"]}
# Calling the tool get_entities with params {"metrics":["revenue"]}

Here are the dimensions you can slice Revenue by, as exposed in the semantic layer. I've grouped them by general category and noted the type when helpful.

Time-based
- metric_time (TIME) with granularities: DAY, WEEK, MONTH, QUARTER, YEAR
- order_id__customer__first_ordered_at (TIME)
- order_id__customer__last_ordered_at (TIME)
- order_id__location__opened_at (TIME)
- order_id__ordered_at (TIME)
- order_item__ordered_at (TIME)

Customer-related (categorical)
- order_id__customer__customer_name
- order_id__customer__customer_type
- order_id__customer__count_lifetime_orders
- order_id__customer__lifetime_spend

Location-related
- order_id__location__location_name
- order_id__location__tax_rate
- order_id__location__opened_at (also listed under time-based)

Product-related
- product__product_name
- product__product_type
- product__is_food_item
- product__is_drink_item
- product__product_price
- product__product_description

Order-related
- order_id__order_total_dim
- order_id__is_food_order
- order_id__is_drink_order
- order_id__ordered_at (also listed under time-based)

Other
- locations (a general/simple dimension representing locations)

Would you like me to run a quick example query slicing Revenue by one of these dimensions (e.g., Revenue by metric_time month, or Revenue by customer_name), so you can see the results? If so, tell me which dimension you want to start with.
```

> User > Between March 2025 and August 2025, give me the revenue by month, by product type and by location name
> 

> Answer:
> 

```
# Calling the tool list_metrics with params {}
# Calling the tool get_dimensions with params {"metrics":["revenue"]}
# Calling the tool get_entities with params {"metrics":["revenue"]}
# Calling the tool query_metrics with params {"metrics":["revenue"],"group_by":[{"name":"metric_time","type":"time_dimension","grain":"MONTH"},{"name":"product__product_type","type":"dimension","grain":null},{"name":"order_id__location__location_name","type":"dimension","grain":null}],"order_by":[{"name":"metric_time","descending":false},{"name":"product__product_type","descending":false},{"name":"order_id__location__location_name","descending":false}],"where":"{{ TimeDimension('metric_time', 'MONTH') }} >= '2025-03-01' AND {{ TimeDimension('metric_time', 'MONTH') }} < '2025-09-01'"}

Here's revenue for 2025, sliced by month, product type, and location (Brooklyn and Philadelphia). Data is pulled from the Revenue metric grouped by:
- metric_time (MONTH)
- product__product_type
- order_id__location__location_name

August 2025
- beverage, Brooklyn: 32,053
- beverage, Philadelphia: 24,703
- jaffle, Brooklyn: 25,365
- jaffle, Philadelphia: 20,258

July 2025
- beverage, Brooklyn: 26,394
- beverage, Philadelphia: 22,915
- jaffle, Brooklyn: 21,655
- jaffle, Philadelphia: 19,432

June 2025
- beverage, Brooklyn: 25,496
- beverage, Philadelphia: 25,032
- jaffle, Brooklyn: 20,639
- jaffle, Philadelphia: 22,516

May 2025
- beverage, Brooklyn: 21,876
- beverage, Philadelphia: 27,374
- jaffle, Brooklyn: 18,010
- jaffle, Philadelphia: 24,128

April 2025
- beverage, Brooklyn: 18,476
- beverage, Philadelphia: 23,774
- jaffle, Brooklyn: 15,582
- jaffle, Philadelphia: 21,414

March 2025
- beverage, Brooklyn: 13,142
- beverage, Philadelphia: 25,130
- jaffle, Brooklyn: 10,760
- jaffle, Philadelphia: 21,186

Would you like this as a CSV or Excel file for download, or should I format it as a pivot-ready table (e.g., months as rows and a column per product/location combination)?
```

OpenAI even suggests generating a CSV so that we could leverage this data in other tools.

## 3. Try it yourself

By connecting an OpenAI agent to dbt Cloud through the dbt MCP server, you've seen how to give an LLM both the context it needs to **discover trusted assets** and the guardrails to **query governed metrics** via the Semantic Layer. This approach keeps answers consistent with your team's definitions, while still letting you ask questions in plain language.

You can adapt the same pattern to other integration options - local or remote MCP, the Responses API, or future ChatGPT integrations - without changing the core logic. With a small amount of setup, you can start embedding these capabilities into internal tools, notebooks, or automations, making it easier for everyone to explore and trust your organization's data.